This document will contain the template and codes for creating master data dictionary , carrying out basic and advanced
text analytics to prepare training set data be be feeeded into an machine learning supervised model to classify
product descriptions into multi level multi class classification/categories. 

Creating the Master Data Dictionary of Product Master Classes (Nouns)
=====================================


In [1]:
# Importing all necessary libraries
import time
start = time.time()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import nltk
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

In [2]:
# Import the dataset contaning the texts showing mapping between all possible  Noun,Modifier and Characteristic
raw_data_catelogue = pd.read_csv('Master_Dictionary.csv')
raw_data_catelogue.drop_duplicates(subset=None, inplace=True)
raw_data_catelogue = raw_data_catelogue.fillna('NA')

In [3]:
raw_data_catelogue.head(10)

,Noun,Modifier,Characteristic,TheValue
0,ABRASIVE,AAAA,NA,NA
1,ABRASIVE,BELT,BOND/BACKING,GLUE BOND
2,ABRASIVE,BELT,BOND/BACKING,RAYON BACKING
3,ABRASIVE,BELT,BOND/BACKING,RESIN BOND
4,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT CLOTH BACKING, CLOSE COATED"
5,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT COTTON BACKING, OPEN COATED"
6,ABRASIVE,BELT,BOND/BACKING,WATERPROOF
7,ABRASIVE,BELT,GRIT,80
8,ABRASIVE,BELT,GRIT,120
9,ABRASIVE,BELT,GRIT,180


In [4]:
import warnings
warnings.filterwarnings('ignore')

defDict = defaultdict(LabelEncoder)
data_dictionary_encoded = raw_data_catelogue.apply(lambda x: defDict[x.name].fit_transform(x))
orig_raw_data_catelogue = data_dictionary_encoded.apply(lambda x: defDict[x.name].inverse_transform(x))

In [5]:
orig_raw_data_catelogue.head(10)

,Noun,Modifier,Characteristic,TheValue
0,ABRASIVE,AAAA,NA,NA
1,ABRASIVE,BELT,BOND/BACKING,GLUE BOND
2,ABRASIVE,BELT,BOND/BACKING,RAYON BACKING
3,ABRASIVE,BELT,BOND/BACKING,RESIN BOND
4,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT CLOTH BACKING, CLOSE COATED"
5,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT COTTON BACKING, OPEN COATED"
6,ABRASIVE,BELT,BOND/BACKING,WATERPROOF
7,ABRASIVE,BELT,GRIT,80
8,ABRASIVE,BELT,GRIT,120
9,ABRASIVE,BELT,GRIT,180


In [6]:
data_dictionary_encoded.head()

,Noun,Modifier,Characteristic,TheValue
0,0,2,616,15012
1,0,54,88,13252
2,0,54,88,16157
3,0,54,88,16314
4,0,54,88,16322


In [7]:
data_dictionary_mapping = pd.concat([orig_raw_data_catelogue,data_dictionary_encoded], axis=1)
headers=['Noun','Modifier','Characteristic','TheValue','Noun_en','Modifier_en','Characteristic_en','TheValue_en']
data_dictionary_mapping.columns=headers

In [8]:
data_dictionary_mapping.head()

,Noun,Modifier,Characteristic,TheValue,Noun_en,Modifier_en,Characteristic_en,TheValue_en
0,ABRASIVE,AAAA,NA,NA,0,2,616,15012
1,ABRASIVE,BELT,BOND/BACKING,GLUE BOND,0,54,88,13252
2,ABRASIVE,BELT,BOND/BACKING,RAYON BACKING,0,54,88,16157
3,ABRASIVE,BELT,BOND/BACKING,RESIN BOND,0,54,88,16314
4,ABRASIVE,BELT,BOND/BACKING,"RESIN BOND, X-WEIGHT CLOTH BACKING, CLOSE COATED",0,54,88,16322


In [10]:
#Saving the dictionary to a file for reference
data_dictionary_mapping.to_csv('data_dictionary_mapping.csv', sep=',')

Basic feature extraction using text data
========================================

In [11]:
import re
manufacturer_ref_nr = pd.read_csv('Manufacture_Ref_No.csv',encoding = "latin")
manufacturer_ref_nr.head()

,Manufacturer,Ref_No,Manufacturer_Name
0,MANUFACTURER_NAME,TAG_NUMBER,SKF
1,MFR,MODEL_NUMBER,BROOK HANSEN
2,MFN,MODEL_NUMBER_OR_TYPE_NUMBER,NIKKISO
3,Manufacture,MN,KUK
4,OEM,EQPT_MNFR_P_N,FORGE ENGG SYSTEM


In [20]:
import re
manufacturer_ref_nr = pd.read_csv('Manufacture_Ref_No.csv',encoding = "latin")

def cleaning_input_text(input_text):
    text = re.sub("[,.]", " ", input_text)
    tokens = nltk.word_tokenize(text)
    all_words_bag_temp_1 = nltk.pos_tag(tokens)
    all_words=[]
    for j in range(len(all_words_bag_temp_1)):
        if(all_words_bag_temp_1[j][1] not in ['JJ','MD','PRP','CD','POS','IN'] and all_words_bag_temp_1[j][0].isalpha()):
            all_words.append(all_words_bag_temp_1[j][0])
    clean_text=' '.join(all_words)
    clean_text_1 = clean_text
    del clean_text
    return clean_text_1

#Fucntion to extract out valid n grams values and characteristics
def ngram_model(text,n_val,char_value):
    from nltk import ngrams
    sentence = text
    n = n_val
    ngrams = ngrams(sentence.split(), n)
    for grams in ngrams:
        str1 = " ".join(list(grams))
        return_val='NA'
        if (str1.upper() == char_value.upper()):
            return_val=char_value.upper()
            return (return_val)
            break
        else:
            return_val='NA'

# Function to extract two word noun
def ngram_model_noun(text,n_val):
    from nltk import ngrams
    sentence = text
    n = n_val
    ngrams = ngrams(sentence.split(), n)
    for grams in ngrams:
        str1 = " ".join(list(grams))
        return_val='NA'
        if (str1.upper() in list(data_dictionary_mapping['Noun'])):
            return_val=str1.upper()
            return (return_val)
            break
        else:
            return_val='NA'
    return return_val

def noun_identification(test_sample):
    from nltk import ngrams
    
    #Cleaning input data
    inp = re.sub(r'^\W*','', test_sample)
    inp = re.sub("[)(]",' ', inp)
    inp = re.sub('\W+',' ', inp)
    word_tokens = nltk.word_tokenize(inp.upper())
    
    #Extracting the first word
    noun = inp.split(' ', 1)[0].upper()

    #Checking if the first word is a noun
    if(noun in data_dictionary_mapping['Noun'].unique()):
        inp_text = ' '.join(word_tokens[:10])
        modifier_list = data_dictionary_mapping[data_dictionary_mapping['Noun'] == noun]['Modifier'].unique()
        for j in modifier_list:
            n_gram = len(j.split())
            modifier_val = ngram_model(inp_text,n_gram,j.upper())
            if (modifier_val != 'NA' and modifier_val is not None):
                dict_noun_modifier = {'noun':noun.upper(),'modifier':modifier_val.upper()}
                return dict_noun_modifier
        if (modifier_val == 'NA' or modifier_val is None):
            dict_noun_modifier = {'noun':noun.upper(),'modifier':'NO MODIFIER'}
            return dict_noun_modifier
    
    #Looking for noun deeper in input text and appending all possible nouns
    elif (noun not in data_dictionary_mapping['Noun'].unique()):
        noun_val=''
        noun_value=''
        modifier_value=''
        inp_text = word_tokens[:10]
        noun_list= data_dictionary_mapping['Noun'].unique()
        noun_list_str = ' '.join(noun_list)
        for k in inp_text:
            if ( re.findall('\\b'+k+'\\b', noun_list_str, flags=re.IGNORECASE) ):
                if(k in list(noun_list) and k not in noun_value):
                    noun_value = str(noun_value) + str(k) + '\n'
        for m in noun_value.split():
            modifier_list = data_dictionary_mapping[data_dictionary_mapping['Noun'] == m]['Modifier'].unique()
            for n in modifier_list:
                if (re.findall('\\b'+n+'\\b', ' '.join(inp_text), flags=re.IGNORECASE)):
                    if (n in list(modifier_list) and n not in modifier_value):
                        modifier_value = str(modifier_value) + str(n) + str('\n')
        n_length = len(noun_value.split())
        m_length = len(modifier_value.split())

        #Check for noun combination if it qualifies for a valid two word noun        
        if (n_length > 1):
            noun_str = noun_value
            noun_str = ' '.join(noun_str.strip().split('\n'))
            noun_new = ngram_model_noun(noun_str,2)
            if (noun_new!='NA'):
                print(noun_new)
                noun_value = noun_new
        
        #Check and assign nouns and modifiers
        if ((n_length > 0) and (m_length>0)):
            dict_noun_modifier = {'noun':noun_value.upper(),'modifier':modifier_value.upper()}
            return dict_noun_modifier
        if ((n_length > 0) and (m_length ==0)):
            dict_noun_modifier = {'noun':noun_value.upper(),'modifier':'NO MODIFIER'}
            return dict_noun_modifier
        if (n_length == 0):
            dict_noun_modifier = {'noun':'NO NOUN','modifier':'NO MODIFIER'}
            return dict_noun_modifier
            
    else:
        dict_noun_modifier = {'noun':'NO NOUN','modifier':'NO MODIFIER'}
        return dict_noun_modifier

def manufacturer(text):
    inp = re.sub("[)(:.]",' ', text).upper()
    manufacturer_tags = manufacturer_ref_nr['Manufacturer'].str.upper().dropna()
    manufacturer_names = manufacturer_ref_nr['Manufacturer_Name'].str.upper().dropna()
    for i in manufacturer_tags:
        if (re.findall('\\b'+i+'\\b', inp, flags=re.IGNORECASE)):
            start_index = inp.index(i)
            word_list = ' '.join(inp[start_index:].split(" ")[0:10])
            for j in manufacturer_names:
                if(re.findall('\\b'+j+'\\b', word_list, flags=re.IGNORECASE)):
                    return(j)

def ref_no(text):
    inp = re.sub("[)(:]",' ', text).upper()
    inp_str=inp.split()
    ref_no_names = manufacturer_ref_nr['Ref_No'].str.upper().dropna()
    ref_no=''
    for i in ref_no_names:
        if (re.findall('\\b'+i+'\\b', inp, flags=re.IGNORECASE)):
            word_list = i.split()
            search = word_list[0]
            if (search in inp_str):
                last_index =inp_str.index(search) + len(word_list)
                if (len(inp_str) > last_index):
                    val = inp_str[inp_str.index(search) + + len(word_list)]
                    if (val not in ref_no):
                        ref_no = str(ref_no) + val + str('\n')
    if (len(ref_no)>0):
        return ref_no
    else:
        return 'NA'

In [26]:
P = manufacturer('GASKET , SPIRAL WOUND   FILLER MATERIAL GRAPHITE SN : QWEB P/N:ABCD \
PRESSURE RATING 300 LB SIZE 7-1/2 ID X 8-1/2 OD STYLE FLEXITALLIC CG \
THICKNESS 1/16 WINDING MATERIAL INCONEL 600 GASKETS FOR FLANGE SIZES > 24 SHALL CONFORM \
ASME B 16.47 FLANGES UNLESS SPECIFIED OTHERWISE.GASKET  FACE  MUST CLEARLY SHOW SIZE, FLANGE RATING AND MANUFACTURER.\
ANSI 900 LBS.NOM. PIPE SIZE : 3/4 IN')
print(P)
#K = ref_no("DEEP EQPT_MNFR_P_N GROOVE BALL BEARING  P/N SINGLE ROW Part Number : s/abcd xyz SPEC: DIN 625 ISO 15  NO.OF SHIELDS: 1  CAGE MATERIAL: PRESSED STEEL  SIZE: 30MM BORE X 62MM OD  WIDTH/HEIGHT: 16MM    MANUFACTURER: SKF MFR.PART NO 6206Z-C3")
K = ref_no("ACCELEROMETER INTRINSICALLY SAFE RANGE: 1 TO 2000HZ SENSITIVITY: 104.3MV/G BIAS LEVEL: 3.2V FOR PUMP MODEL NO: 12ECR-242 MANUFACTURER: PCB PIEZOTRONICS MFR.MODEL NO: 351M12 MFR.SERIAL NO.3468 / 2961 EQUIPMENT MFR: EBARA EQUIP.MFR.PART NO: 8070143-01")
print(K)
# m = noun_identification('ELECTRIC MOTOR \
# TYPE: INDUCTION INTEGRATED\
# FRAME: ADH-100LB\
# VOLTS: 415V\
# KW: 2.2\
# RPM: 1440\
# MANUFACTURER: BROOK HANSEN\
# MFR.MODEL NO: ADF-100LB"')
# 
# m = noun_identification('ANGULAR  BALL BEARING SIZE: 65MM X 140MM \
# WIDTH/HEIGHT: 33MM \
# CAGE MATERIAL: STEEL \
# TANDEM & SINGLE ASSY \
# SINGLE ROW 1 SET=3PCS \
# FOR CENTRIFUGAL PUMP MANUFACTURER: NTN OR SKF \
# SKF.PART NO: 7313-BECBJ \
# NTN.PART NO: 7313-BDT \
# EQUIPMENT MFR: SHIN NIPPON \
# EQUIP.MODEL NO: 2.5X14SVH-A8 ')
m=noun_identification('O-RING INSIDE DIAMETER: 196.22MM(7.725IN) OUTSIDE DIAMETER: 210.22MM(8.276IN) THICKNESS: 7.00MM(0.276IN) MATERIAL: VITON DASH NO: 444 HARDNESS: 80 SH A TEMPERATURE RANGE: -20 TO +200°C TYPE: BEWZ19/4000 FOR HEAT EXCHANGER MANUFACTURER: ROBOX MFR.ORIGINAL PART NO: P1602-K03-505 MFR.REVISED PART NO: 3AV71962269B MFR.DRAWING NO: V-2158-207-A-009,POS.NO: 505 EQUIPMENT MFR: ELTACON EQUIP.MODEL NO: 10-K101A-E002 THE ITEM NEEDS TO BE SUPPLIED WITH THE FOLLOWING: -PRODUCTION DATE -EXPIRY DATE -MAXIMUM SHELF LIFE NOTE: THE SUPPLIED ITEM SHOULD HAVE 80% SHELF LIFE')
#m=noun_identification('O-RING INSIDE DIAMETER: 24.99MM(0.984IN) OUTSIDE DIAMETER: 32.05MM(1.262IN) THICKNESS: 3.53MM(0.139IN) DASH NO: 214 FOR BALL VALVE,SIZE: 3IN X 2IN,CL900 VALVE BODY TYPE: SPLIT BODY MANUFACTURER: RING-O-VALVE MFR.ORIGINAL PART NO: 613002400009 MFR.REVISED PART NO: 9F-613002400009 POS.NO: 23 EQUIP.MODEL NO: GBAX2-3-900RTJ EQUIP.SERIAL NO: 96/3789')
#m=noun_identification('FLOATING BALL VALVE BODY MATERIAL: ASTM A216 WCB BALL MATERIAL: SS316 STEM MATERIAL: SS316 SEAT MATERIAL: PTFE BODY SEAL MATERIAL: TFM STEM SEAL MATERIAL: PTFE/TFM PACKING MATERIAL: GRAPHITE+PTFE BOLTING MATERIAL: ASTM A193 B7, A194 2H LIVE LOADED SINGLE PACKING END CONNECTION: THREADED NPTF FACE TO DIMENSION: 114MM OPERATION: HAND LEVER OPERATED MAX.SHUT OFF PRESSURE: 900PSI @ 100°F 100PSI @ 350°F WSP: 150PSI COLD WORKING PRESSURE 2000 INSPECTION AS PER API 598 PIPE THREADS AS PER ANSI B1.20.1 MATERIAL TEST CERTIFICATE: EN 10204-3.1 TO BE SUPPLIED WITH ALL TEST CERTIFICATES VALVE CONSTRUCTION CONFORMS TO MSS SP25 PAINTING SPEC AS PER: RG-S-00-13A0-004 PAINTING SYSTEM NO: 1C AS PER  VALVE SPEC: RG6-S-00-1360-085 PRESSURE RATING: CL150 SIZE: 1IN MANUFACTURER: JAMESBURY (OR) EQUIVALENT MFR.PART NO: 1 A2236TTD EQUIPMENT.MFR: GE EQUIP.MFR.PART NO: 275A8828P002 EQUIP.MODEL NO: MS-6001-B')
#m = noun_identification('@#$%^ FLANGE FOR CONCENTRIC EDGE ORIFICE TAG NO:40352 MODEL RAISED FACE TYPE ORIFICE FLANGE-600 NOMINAL PIPE SIZE:600mm(24"):COMBINE WITH ORIFICE PLATE MODEL:FOR MODEL FOP FLANGE TAP MOUNTING:WELD NECK, FLANGE MATERIAL: CARBON STEEL-150, FLANGE RATING:ANSI CLASS 150-1993,GASKET MATERIAL:NONASBESTOS, BOLT MATERIAL: SNB7G4, NUT MATERIAL:ASTM A193 Gr4/NN, TAP CONNECTION: 1/2 NPT FEMALE, MAKE; NEWFLOW DATA SHEET ATTACHED')
#m = noun_identification('GASKET,SPIRAL WOUND,RWI-RJ,2,1500#_____PO TEXT_____FOR REPLACEMENT OF RING JOINT GASKET TYPE                 : SPIRAL WOUND STYLE                : RWI-RJ SIZE                 : 2"""" RATING               : 1500# DESIGN SPECIFICATION : ASME B.16.20 INNER RING MATERIAL  : SS304 FILLER               : GRAPHITE OUTER RING  -        : CARBON STEEL_____CLASS_____GASKET_____CHARACTERISTICS_____   ADDITIONAL_INFORMATION - 1500#   , MAT_GASKET - SOFT IRON   , MAT_GRADE - ASME16.20   , PACKAGE_NUMBER - PEM/113/GEN   , SIZE - 2""""   , TYPE - SPIRAL WOUNDMANUFACTURER INFO: GARLOCK/91311-5102')
#m = noun_identification('AIR CIRCUIT BREAKER AEROMAT TYPE: WITHDRAW NO OF POLES: 3 CURRENT: 1000A')
#m = noun_identification('PUSH BUTTON  ILLUMINATED  FAULT; TYPE: 1A + 1B; SIZE: 25MM DIA; COLOR: ORANGE; VOLTAGE RATING: 120VAC; MANUFACTURER NAME: YONGSUNG ELECTRIC CO; MANUFACTURER PART NO: YSNPBL2-AL11O')
#m = noun_identification('ROUND BOLT SET')
print(m)

PIPE
8070143-01
/
351M12

{'noun': 'RING\n', 'modifier': 'NO MODIFIER'}


In [ ]:
'''
Testing Module to spool out test results
Step 1 : Change the Test file name 
Step 2 : Change the output file name
Step 3 : Compile and run

'''

def testing_module(choice,test_input):
    if (choice == '1'):
        df_raw_test = pd.read_csv("Test_Data.csv",encoding = "latin")
        df_raw_test.reset_index(drop=True, inplace=True)
        output_df = pd.DataFrame(columns=['Prod_ID','Material_Desc','Noun','Modifier'])
        counter1=df_raw_test.shape[0]
        
        # Extracting correct noun and modifier pair
        for i in range(counter1):
            if(i%1 == 0):
                print(i,' Inside 1 if loop')
            prod_id = int(1000 + i)
            noun_modifier_dict = noun_identification(df_raw_test['Long_Desc'][i].replace(',',' '))
            noun_value=noun_modifier_dict['noun'].strip('\n')
            modifier_value = noun_modifier_dict['modifier'].strip('\n')
            concat = noun_value + " | " + modifier_value
            output_df = output_df.append({'Prod_ID':prod_id,'Material_Desc':df_raw_test['Long_Desc'][i],'Noun':noun_value,'Modifier':modifier_value},ignore_index=True)

        out_df_final = pd.DataFrame()
        output_char_value_df_temp = pd.DataFrame()
        df1 = pd.DataFrame()
        counter = df_raw_test.shape[0]
        dummy_data = {'Dummy': [np.NaN]}
        output_char_value_df_dummy = pd.DataFrame(data=dummy_data)
        
        #Extracting attribute and value pair for a noun-modifier combination
        for i in range(counter):
            if(i%100 == 0):
                print(i,'Inside 2 if loop')
            noun = output_df['Noun'][i].strip('\n')
            modifier = output_df['Modifier'][i].strip('\n')
            char_list = data_dictionary_mapping[(data_dictionary_mapping['Noun'] == noun) & (data_dictionary_mapping['Modifier'] == modifier) ]
            char_values_all = char_list['Characteristic'].unique()
            output_char_value_df = pd.DataFrame(columns=char_values_all)
            
            #Loop to check if the noun is a single noun.we extract only attribute-value for these kind's of noun.Multi noun
            #are to be evaluated by subject matter expert
            if(len(noun.strip().split('\n')) == 1):
                counter1 = len(char_values_all)
                
                #Loop to scan all attributes of a noun-modifier pair
                for j in range(counter1):
                    char_value = char_values_all[j]
                    values = char_list[char_list['Characteristic'] == char_value]['TheValue']
                    
                    #Loop to scan all values of a noun-modifier-characteristic pair
                    for k in values:
                        n_gram = len(k.split())
                        #N-gram model looks for all the values in the material description
                        char_flag = ngram_model(output_df['Material_Desc'][i],n_gram,k)
                        
                        #If the n-gram model yields no results we go for manual searching the attribute and extracting the 
                        #values corresponding to it.
                        if char_flag is None:
                            word_list=char_value.split()
                            search = word_list[0]
                            prod_desc=output_df['Material_Desc'][i]
                            clean_string = prod_desc.replace(':', ' ').replace('-', ' ')
                            list_of_words = clean_string.split()
                            if (search in list_of_words):
                                last_index =list_of_words.index(search) + len(word_list)
                                if (len(list_of_words) > last_index):
                                    char_flag = list_of_words[list_of_words.index(search) + len(word_list)]
                                    output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                        if (char_flag is not None):
                            output_char_value_df=output_char_value_df.append({char_value:char_flag},ignore_index=True)
                        else:
                            output_char_value_df=output_char_value_df.append({char_value:'NA'},ignore_index=True)
                
                #The for-loop for value extraction results in multi row with NaN's.We are intrested in the last row after 
                #all the values have been extracted successfully
                output_char_value_df = output_char_value_df.fillna(method='ffill')
                output_char_value_df = output_char_value_df.tail(1)
            
            #Creating data for multi noun records.Inserting dummy column name with NaN.This is required to maintain consistency
            #in formats for display.Materials with multiple noun will not have any attribute value pair
            
            if (output_char_value_df.empty):
                col = output_char_value_df_dummy.columns
                for k in range(output_char_value_df_dummy.shape[1]):
                    col_name = col[k]
                    output_char_value_df.loc[0,col_name] = np.NaN
            
            #Piling all materail descriptions and storing them in a temporary dataframe
            output_char_value_df_temp = output_char_value_df_temp.append(output_char_value_df)
            
            #Deleting the dataframe data for next loop run for new material decription
            output_char_value_df=output_char_value_df.iloc[0:0]
        
        output_char_value_df_temp.reset_index(drop=True, inplace=True)
        output_df.reset_index(drop=True, inplace=True)
        
        #Creating dataframe to store Prod_ID,Material_Desc,Noun,Modifier,Attribute and Values
        final_output_df = pd.concat([output_df,output_char_value_df_temp],axis=1)
        
        #Creating a dataframe with only attribute and value for short and long description extraction
        new = pd.DataFrame()
        new = final_output_df.iloc[:,5:]
        df_temp1 = final_output_df.loc[:,'Prod_ID':'Modifier']
        df = pd.DataFrame()
        
        #Short and Long description derivation
        for i in range(new.shape[0]):
            if(i%100 == 0):
                print(i,'Inside 3 if loop')       
            short_desc=''
            long_desc=''
            val=''
            for j in new.columns:
                val = final_output_df.loc[i,j]
                if (val != 'NA'):
                    if(not pd.isnull(val)):
                        short_desc = str(short_desc) + str(val) + ','
                        long_desc =  str(long_desc) + str(j) + ' : ' + str(val) + '\n'
            df = df.append({'short_description':short_desc,'long_description':long_desc},ignore_index=True)
        
        #Formattig noun and modifier names represenation for long and short description
        
        df_formatter=pd.concat([df_temp1,df],axis=1)
        modifier_dict = pd.read_csv('modifier_abbreviation_dict.csv')
        modifier_dict.reset_index(drop=True, inplace=True)
        for i in range(df_formatter.shape[0]):
            if(i%100 == 0):
                print(i,'Inside 4 if loop')
            if(len(df_formatter.loc[i,'Noun'].split('\n')) == 1):
                new_long_desc = "*** " + str(df_formatter.loc[i,'Noun'].strip('\n')) + ' , ' + str(df_formatter.loc[i,'Modifier'].strip('\n')) + " *** " + '\n\n' + str(df_formatter.loc[i,'long_description'])
                modi_search = df_formatter.loc[i,'Modifier']
                if(modi_search in list(modifier_dict['Modifier'])):
                    short_modifier =modifier_dict[modifier_dict['Modifier'] == modi_search ]['Abbreviation'].values[0]
                    new_short_desc =  str(df_formatter.loc[i,'Noun']) + ' , ' + str(short_modifier) + ': ' + str(df_formatter.loc[i,'short_description'])
                    new_short_desc = new_short_desc[:-1]
                else:
                    new_short_desc = str(df_formatter.loc[i,'Noun'].strip('\n')) + ',' + str(df_formatter.loc[i,'Modifier'].strip('\n')) + ':' + str(df_formatter.loc[i,'short_description'])
                    new_short_desc = new_short_desc[:-1]
                df_formatter.loc[i,'long_description'] = new_long_desc
                df_formatter.loc[i,'short_description'] = new_short_desc
        
        # Manufacturer , Ref_No and Exception column extraction
        
        df_exception = pd.DataFrame()
        df_manufacture = pd.DataFrame()
        df_ref_no = pd.DataFrame()
        manufacturer_ref_no_tags= ' '.join(manufacturer_ref_nr['Manufacturer'].append(manufacturer_ref_nr['Ref_No']).dropna())

        for k in range(counter):
            if(k%100 == 0):
                print(k,'Inside 5 if loop')
            #Manufacturer
            
            manufacturer_val = manufacturer(final_output_df.loc[k,'Material_Desc'])
            if (manufacturer_val != 'NA' and manufacturer_val is not None):
                df_manufacture = df_manufacture.append({'Manufacturer': manufacturer_val},ignore_index=True)
            else:
                df_manufacture = df_manufacture.append({'Manufacturer': 'NA'},ignore_index=True)
            
            #Ref_No
            ref_no_val = ref_no(final_output_df.loc[k,'Material_Desc'])
            if (ref_no_val != 'NA' and ref_no_val is not None):
                df_ref_no = df_ref_no.append({'Ref_no': ref_no_val},ignore_index=True)
            else:
                df_ref_no = df_ref_no.append({'Ref_no': 'NA'},ignore_index=True)

            #Creating a exception column to dump all the text information which do not belong to a noun,modifier,atttribute 
            #or values

            mat_desc=df_temp1.loc[k,'Material_Desc'].replace(',',' ') 
            meaningful_text = df_formatter.loc[k,'long_description'] + ' ' + str(df_manufacture.loc[k,'Manufacturer']) + ' ' + str(df_ref_no.loc[k,'Ref_no']) + ' ' + manufacturer_ref_no_tags
            meaningful_text = meaningful_text.upper()
            exception = [w.upper() for w in nltk.word_tokenize(mat_desc) if w not in nltk.word_tokenize(meaningful_text)]
            exception = ' '.join(exception)
            if (len(exception)>0):
                df_exception = df_exception.append({'exception': exception},ignore_index=True)
            else:
                df_exception = df_exception.append({'exception': 'No exception found'},ignore_index=True)
        
        #Consolidation all intermediat dataframes and creating final dataframe with below columns
        
        '''
        PROD_ID
        MATERIAL_DESC
        NOUN
        MODIFIER
        LONG_DESCRIPTION
        SHORT_DESCRIPTION
        MANUFACTURER
        REF_NO
        EXCEPTION
        LIST OF ATTRIBUTES
        
        '''

        output_dataframe=pd.DataFrame()
        new_output_df=pd.DataFrame()
        new_output_df=final_output_df.iloc[:,4:]
        output_dataframe = pd.concat([df_formatter,df_manufacture,df_ref_no,df_exception,new_output_df],axis=1)
        
        #Dropping the dummy column created for multi noun records
        if ('Dummy' in output_dataframe.columns):
            output_dataframe.drop(['Dummy'],axis=1,inplace=True)
        output_dataframe.to_csv('final_output_df_file_import18.csv')
        display(output_dataframe)
        print('Please check you local file system for output generated : File ----> final_output_df_file_import18.csv')
        return output_dataframe


In [ ]:
# import warnings
warnings.filterwarnings('ignore')

import time
start = time.time()

choice = input("Enter choice(1/2):")
if choice == '1':
    print('Make sure you have feeded your input in the Test_Data.csv')
    test_input=''
    pred_output_df = pd.DataFrame()
    pred_output_df = testing_module(choice,test_input.upper())
    pred_output_df = pred_output_df.sort_values(['Noun','Modifier'], ascending=[True,True])
if choice == '2':
    #test_input = input('Enter your test sample\n')
    test_input = "SPIRAL WOUND GASKET SIZE: 1""X 900/1500 LBS;WITH EXTERNAL GUIDE RING (STYLE-CG), WINDING MATERIAL:316L FILLER MATERIAL GRAPHITE AS PER ASME B 16.20 TO FIT FOR ANSI B 16.5 FLANGES"
    pred_output_df = testing_module(choice,test_input.upper())
    #display(pred_output_df)
end = time.time()
temp = end-start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
print(hours,':',minutes,':',seconds)

In [ ]:
pred_output_df = pred_output_df.sort_values(['Noun','Modifier'], ascending=[True,True])
n_noun_modifier= pd.DataFrame()
n_noun_modifier = pred_output_df.loc[:,'Noun':'Modifier'].drop_duplicates()
n_noun_modifier.reset_index(drop=True, inplace=True)
writer = pd.ExcelWriter('final_output_df_file_import_excel18.xlsx', engine='xlsxwriter')
for i in range(n_noun_modifier.shape[0]):
    noun=n_noun_modifier.loc[i,'Noun'].replace('/','_')
    modifier=n_noun_modifier.loc[i,'Modifier'].replace('/','_')
    df_name = str(noun) + "_" + str(modifier)
    name = n_noun_modifier.loc[i,'Noun'].replace(' ','_').replace('/','_') + "_" + n_noun_modifier.loc[i,'Modifier'].replace(' ','_').replace('/','_')
    name = name[ 0 : 0 + 30]
    df_taxonomy = pd.DataFrame()
    df_taxonomy = pred_output_df[pred_output_df['Noun'] == noun]
    df_taxonomy = df_taxonomy[df_taxonomy['Modifier'] == modifier]
    df_taxonomy.dropna(axis='columns',inplace=True,how='all')
    if ('NA' in df_taxonomy.columns):
        df_taxonomy.drop(['NA'],axis=1,inplace=True)
    df_taxonomy = df_taxonomy.replace('NA','')
    df_taxonomy.reset_index(drop=True, inplace=True)
    df_taxonomy.columns = map(str.upper, df_taxonomy.columns)
    workbook  = writer.book
    display(df_taxonomy)
    for i in range(df_taxonomy.shape[0]):
        if('EXCEPTION' in df_taxonomy.columns):
            col_names_omit=' '.join(df_taxonomy.columns)
            mat_desc = df_taxonomy.loc[i,'EXCEPTION']
            exception = [w.upper() for w in nltk.word_tokenize(mat_desc) if w not in nltk.word_tokenize(col_names_omit)]
            exception = ' '.join(exception)
            df_taxonomy['EXCEPTION']=exception
    df_taxonomy.to_excel(writer,sheet_name=name)
    worksheet = writer.sheets[name]
    start_row = 1
    start_col = 7
    end_row = df_taxonomy.shape[0]
    end_col = df_taxonomy.shape[1]
    format0 = workbook.add_format({'bg_color': '#FFA07A','text_wrap': True})
    format1 = workbook.add_format({'bg_color': '#90EE90','text_wrap': True})
    format2 = workbook.add_format({'font_color': '#FFFFFF','bg_color': '#0000FF','text_wrap': True})
    cell_format = workbook.add_format({'text_wrap': True})
    cell_format.set_align('left')
    cell_format.set_align('top')
    worksheet.conditional_format(1, 0, end_row, end_col, {'type':'blanks', 'format': format0})
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'type':'no_blanks', 'format': format1})
    worksheet.conditional_format(start_row, start_col, end_row, end_col, {'format': cell_format})
    worksheet.conditional_format(0, 0, 0, end_col, {'type':'no_blanks', 'format': format2})
writer.save()